In [ ]:
#  Importamos los módulos que necesitaremos para realizar el ejercicio
from qiskit import QuantumCircuit, Aer
from qiskit.visualization import plot_histogram
from numpy.random import randint
import numpy as np

## Paso 1

In [ ]:
# Inicializamos una distribución aleatoria
np.random.seed(seed=0)

n = 100 # Largo de la cadena de bits 

#  Generamos la cadena de bits aleatorios k de Alice
alice_bits = randint(2, size=n)

#  Generamos la cadena de bit aleatorios b de Alice
alice_bases = randint(2, size= n)

print(alice_bases)

## Paso 2 

In [ ]:
def alice_prepare_qubit(a_bits, a_bases):
    ''' 
    Inputs:
        a_bits  (list) : bit string k
        a_bases (list): bit string b
    Outputs:
        qubit   (list): contains the circuit that generates each Alice's 
                state according k and b.
    ''' 
    qubit = []
    for i in range(n):
        qc = QuantumCircuit(1,1)
    # ------------------------------------------------
        # Prepara los qubits en la base Z
        if a_bases[i] == 0:
            if a_bits[i] == 0:
                pass 
            else:
                qc.x(0)
        # Prepara los qubits en la base X
        else: 
            if a_bits[i] == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
    # ------------------------------------------------
        qc.barrier()
        qubit.append(qc)
    return qubit

In [ ]:
#Aplicamos la función recien creada a todos los bits aleatorios de Alice
protocol = alice_prepare_qubit(alice_bits, alice_bases)

## Paso 3

In [ ]:
#  Generamos la cadena de bit aleatorios b tilde de Bob
bob_bases = randint(2, size = n)
print(bob_bases)

Ahora deberás completar la función `measure_qubit` que aplica la medida de Bob a través de una simulación y almacena los resultados en la lista `measurements`. 

In [ ]:
def measure_qubit(circuit, bases):
    '''
    Inputs:
        circuit      (list) : contains the qubits sent by Alice
        bases        (list) : bit string b tilde
    Outputs:
        measurements (list) : bit string k tilde
    '''
    measurements = []
    for i in range(n):
# ------------------------------------------------  
        if bases[i] == 0: # medimos en la base Z
            circuit[i].measure(0,0)
        if bases[i] == 1: # medimos en la base X
            circuit[i].h(0)
            circuit[i].measure(0,0)
# ------------------------------------------------         
        aer_sim = Aer.get_backend('aer_simulator')
        result = aer_sim.run(circuit[i], shots=1024, memory=True).result()
        measured_bit = int(result.get_memory()[0])
        measurements.append(measured_bit)
    return measurements

Al aplicar la función, obtenemos el siguiente circuito

In [ ]:
#Medicion de Bob
bob_results = measure_qubit(protocol, bob_bases)

## Paso 4

In [ ]:
def remove_garbage(a_bases, b_bases, bits):
    '''
    Inputs:
        a_bases   (list) : bit string b
        b_bases   (list) : bit string b tilde
        bits      (list) : bits that we keep or discard
    Outputs:
        good_bits (list): contains the bits we keep
    '''
    good_bits = []
#--------------------------------------------------------
    for i in range(n):
        if a_bases[i] == b_bases[i]:
            # If both used the same basis, add
            # this to the list of 'good' bits
            good_bits.append(bits[i])
 #--------------------------------------------------------           
    return good_bits

De esta forma, Alice y Bob descartan los bits que no utilizarán, quedando así los bits que formarán parte de la clave secreta

In [ ]:
#Alice y Bob remueven los qubit que no se van a utilizar
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
print(alice_key)

bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
print(bob_key)

## Paso 5 

In [ ]:
def sample_bits(bits, selection):
    '''
    Inputs:
        bits      (list) : bit string
        selection (list) : bits that we select from the list "bits"
    Outputs:
        sample    (list) : sample bits to compare
    '''
    sample = []
    for i in selection:
        # usamos np.mod para asegurarnos que el bit que escibimos siempre
        # esté en la lista de rango:
        i = np.mod(i,len(bits))
        # pop(i) elimina el elemento de la lista con índice i
        sample.append(bits.pop(i))
    return sample

Notar que Alice y Bob hacen pública esta muestra, pero como los bits ya no son secretos, no serán parte de la clave

In [ ]:
# Alice y Bob comparan algunos bits para corroborar de que el protocolo funcionó

# Tamaño de la muestra
sample_size = 15

# Bits a seleccionar
bit_selection = randint(len(alice_key), size=sample_size)

# Definimos la muestra
bob_sample = sample_bits(bob_key, bit_selection)
alice_sample = sample_bits(alice_key, bit_selection)

print("  bob_sample = " + str(bob_sample))
print("alice_sample = "+ str(alice_sample))

In [ ]:
print(bob_key)
print(alice_key)
print("El largo de la llave es %i" % len(alice_key))

# **Ejercicio 2: Eve al ataque**

In [ ]:
np.random.seed(seed=3)
alice_bits = randint(2, size= n)
alice_bases = randint(2, size= n)
protocol = alice_prepare_qubit(alice_bits, alice_bases)

### ¡Ataque!

Eve intercepta los qubits que Alice envía públicamente y los mide con una selección aleatoria de bases, de la misma manera que Bob lo hará más adelante. 

In [ ]:
# Generamos la cadena bits aleatorios de Eve
eve_bases = randint(2, size = n)

# Eve mide sujeto a eve_bases
intercepted_message = measure_qubit(protocol, eve_bases)

print(intercepted_message)

## Paso 3

In [ ]:
bob_bases = randint(2, size = n)
bob_results = measure_qubit(protocol, bob_bases)
display(protocol[0].draw(output="mpl"))
aer_sim = Aer.get_backend('aer_simulator')
job = aer_sim.run(protocol[0])
plot_histogram(job.result().get_counts())

## Paso 4

Bob y Alice revelan las bases que seleccionaron y descartan los bits que no usaran

In [ ]:
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)

## Paso 5

Bob y Alice comparan la misma selección aleatoria de sus claves para ver si su mensaje fue interceptado

In [ ]:
sample_size = 15
bit_selection = randint(n, size=sample_size)
bob_sample = sample_bits(bob_key, bit_selection)
alice_sample = sample_bits(alice_key, bit_selection)

print("  bob_sample = " + str(bob_sample))
print("alice_sample = "+ str(alice_sample))
print("Is alice_key equal to bob_key? " + str(bob_sample == alice_sample)) 

¡La clave de Bob y Alice no coinciden! Sabemos que esto es porque Eve intentó obtener información de clave entre el paso 2 y 3, lo que cambió los estados de los qubits. Sin embargo, Alice y Bob podrían pensar que se debió al ruido en el medio que enviaron los qubits. De todas maneras, ellos descartarán esta clave y repetirán el protocolo de nuevo. El intento de intercepción de Eve ha fallado.


# **Análisis del Riesgo**

Para este tipo de intercepción, en el cual Eve mide todos los qubits, hay una pequeña posibilidad de que la muestra (sample) de Alice y Bob coincidad, y que Alice envié su mensaje con la presencia del espía malicioso, Eve. Vamos a calcular esta posibilidad y ver cuán arriesgado es hacer distribución de claves cuánticas.

                    IMAGEN

Si Alice y Bob utilizan 1 bit para la muestra, la probabildad de que sus claves coincidan en presencia de un espía es igual a $0.75$. Si comparan 2 bits, la probabilidad decrece a $0.75^2 = 0.5625$. Podemos ver que la probabilidad de que Eve pase desapercibida decrece a medida que Alice y Bob aumentan el largo de su muestra, tal que

$ P_\text{indetectada} = 0.75^f$

donde $f$ es el largo de la muestra.

Si decidimos comparar 15 bits como hicimos anteriormente, hay un $1.3\%$ de posibilidades de que Eve no sea detectada. Si esto nos parece demasiado arriesgado, podríamos comparar 50 bits en su lugar, y tendríamos un $0.00006\%$ de posibilidades de ser espiados sin saberlo.

# **Encriptación del Mensaje**

Una vez que la clave fue distribuida, Alice puede encriptar su mensaje usando la técnica one-time pad: ella simplemente suma los bits de la clave con los que ella quiere enviar. Por lo que si ella posee una clave $c$ y su mensaje lo codifica en una cadena de bits $m$, el mensaje encriptado será $e = m \oplus c \; \text{mod} \; 2$. Bob podrá desencriptar el mensaje añadiendo su clave al mensaje encriptado, tal que $m = e \oplus c \; \text{mod} \; 2$.

In [ ]:
np.random.seed(seed=4)
n = 100
# Paso 1
alice_bits = randint(2, size=n)
alice_bases = randint(2, size=n)
# Paso 2
protocol = alice_prepare_qubit(alice_bits, alice_bases)
# Ataque!
eve_bases = randint(2, size=n)
intercepted_qubits = measure_qubit(protocol, eve_bases)
# Paso 3
bob_bases = randint(2, size=n)
bob_results = measure_qubit(protocol, bob_bases)
# Paso 4
bob_key = remove_garbage(alice_bases, bob_bases, bob_results)
alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
# Step 5
sample_size = 10 # Cambia esto a algo más bajo y mira si Eve
                 # puede interceptar el mensaje sin que Alice
                 # y Bob lo sepan
bit_selection = randint(n, size=sample_size)
bob_sample = sample_bits(bob_key, bit_selection)
alice_sample = sample_bits(alice_key, bit_selection)

if bob_sample != alice_sample:
    print("La interferencia de Eve fue detectada.")
else:
    print("Eve no fue detectada")